# Fase 0: Selecionando apenas as linhas com o valor Strand positivo (+)

Aqui nos queremos selecionar de cada arquivo .gff3 as linhas em que o valor da coluna Strand seja '+'. Selecionadas essas linhas, queremos recuperar os cromossomos Chr (seqid = indica a qual cromossomo o elemento transponível pertence) e os valores da coluna Start (posição inicial do elemento dentro desse cromossomo) e End (posição final). No fim, queremos apenas as linhas com Strand positivo e seus valores em 'seqid', 'start' e 'end'.

In [11]:
import pandas as pd
import glob
import os

# Caminhos das pastas
input_dir = '../data/arquivos_TEAnnotation_brutos/'
output_dir = '../data/arquivos_TEAnnotation_filtrados/'
os.makedirs(output_dir, exist_ok=True)

# Nomes padrão das colunas GFF3
column_names = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']

print("Iniciando Fase 0: Filtrando arquivos .gff3...")

# Lista todos os arquivos .gff3
gff_files = glob.glob(os.path.join(input_dir, '*.gff3'))

if not gff_files:
    print(f"Nenhum arquivo .gff3 encontrado em '{input_dir}'!")
else:
    for file_path in gff_files:
        filename = os.path.basename(file_path)
        print(f"\nProcessando: {filename}")
        
        # Ler o arquivo
        df = pd.read_csv(file_path, sep='\t', comment='#', header=None, names=column_names)
        
        # Converter para numérico e limpar dados inválidos
        df['start'] = pd.to_numeric(df['start'], errors='coerce').astype('Int64')
        df['end'] = pd.to_numeric(df['end'], errors='coerce').astype('Int64')
        df = df.dropna(subset=['start', 'end'])

        # Filtrar apenas as linhas com strand positivo
        df_filtrado = df[df['strand'] == '+'][['seqid', 'start', 'end']]

        # Caminho de saída
        output_path = os.path.join(output_dir, filename.replace('.gff3', '_filtrado.tsv'))
        
        # Salvar resultado
        df_filtrado.to_csv(output_path, sep='\t', index=False)
        
        print(f"Linhas filtradas: {len(df_filtrado)}")
        print(f"Salvo em: {output_path}")

print("\nFase 0 concluída com sucesso! Arquivos filtrados prontos para uso nas próximas etapas.")


Iniciando Fase 0: Filtrando arquivos .gff3...

Processando: TEAnnotationFinal_Helitron.gff3
Linhas filtradas: 24769
Salvo em: ../data/arquivos_TEAnnotation_filtrados/TEAnnotationFinal_Helitron_filtrado.tsv

Processando: TEAnnotationFinal_SINE.gff3
Linhas filtradas: 3755
Salvo em: ../data/arquivos_TEAnnotation_filtrados/TEAnnotationFinal_SINE_filtrado.tsv

Processando: TEAnnotationFinal_LINE.gff3
Linhas filtradas: 11174
Salvo em: ../data/arquivos_TEAnnotation_filtrados/TEAnnotationFinal_LINE_filtrado.tsv

Processando: TEAnnotationFinal_TIR.gff3
Linhas filtradas: 83849
Salvo em: ../data/arquivos_TEAnnotation_filtrados/TEAnnotationFinal_TIR_filtrado.tsv

Processando: TEAnnotationFinal_LTR.gff3
Linhas filtradas: 191815
Salvo em: ../data/arquivos_TEAnnotation_filtrados/TEAnnotationFinal_LTR_filtrado.tsv

Processando: TEAnnotationFinal_MITE.gff3
Linhas filtradas: 26350
Salvo em: ../data/arquivos_TEAnnotation_filtrados/TEAnnotationFinal_MITE_filtrado.tsv

Fase 0 concluída com sucesso! Arquivo

# Imports e Definições de Caminhos (preparação para a Fase 1)

In [12]:
from Bio import Entrez
from Bio import SeqIO

# 1. Caminhos dos arquivos filtrados (.tsv)
input_dir = '../data/arquivos_TEAnnotation_filtrados/'

# 2. Pasta para onde baixaremos os genomas FASTA
fasta_dir = '../data/fasta_genomas/'
os.makedirs(fasta_dir, exist_ok=True)

# 3. Nome do arquivo de saída final
output_dataset = '../data/TE_dataset_final.csv'

# 4. Email para o NCBI (para usar a API)
Entrez.email = "luizgcorreiadosantos@usp.br"

# Fase 1: Consolidar Coordenadas (Pandas)

In [13]:
print("Iniciando Fase 1: Consolidando arquivos TSV...")

all_data = []

# Encontrar todos os seus arquivos .tsv filtrados
tsv_files = glob.glob(os.path.join(input_dir, '*.tsv'))

if not tsv_files:
    print(f"ERRO: Nenhum arquivo .tsv encontrado em '{input_dir}'")
    print("Por favor, execute o script de filtragem primeiro.")
else:
    for file_path in tsv_files:
        # 1. Ler o arquivo de coordenadas
        df = pd.read_csv(file_path, sep='\t')
        
        # 2. Extrair o nome da Classe do nome do arquivo
        # Ex: 'TEAnnotationFinal_TIR_filtrado.tsv' -> 'TIR'
        filename = os.path.basename(file_path)
        classe = filename.replace('TEAnnotationFinal_', '').replace('_filtrado.tsv', '')
        
        # 3. Adicionar a coluna "Classe"
        df['Classe'] = classe
        
        all_data.append(df)

    # 4. Combinar tudo em um DataFrame
    coords_df = pd.concat(all_data, ignore_index=True)

    print(f"Total de {len(coords_df)} coordenadas de TEs carregadas.")
    print("Classes encontradas:")
    print(coords_df['Classe'].value_counts())
    print(coords_df.head())

Iniciando Fase 1: Consolidando arquivos TSV...
Total de 341712 coordenadas de TEs carregadas.
Classes encontradas:
Classe
LTR         191815
TIR          83849
MITE         26350
Helitron     24769
LINE         11174
SINE          3755
Name: count, dtype: int64
  seqid      start        end    Classe
0     7   82856122   82857584  Helitron
1     1  239302471  239302834  Helitron
2     8    2261318    2261604  Helitron
3     5  124665404  124665608  Helitron
4     9   77998293   78002783  Helitron


# Fase 2: Baixar Genomas (BioPython.Entrez)

In [ ]:
print("\nIniciando Fase 2: Baixando genomas do NCBI...")

# Mapeamento do Cromossomo ('seqid') para o ID de Acesso do GenBank
chr_accessions = {
    '1': 'LR618874.1',
    '2': 'LR618875.1',
    '3': 'LR618876.1',
    '4': 'LR618877.1',
    '5': 'LR618878.1',
    '6': 'LR618879.1',
    '7': 'LR618880.1',
    '8': 'LR618881.1',
    '9': 'LR618882.1',
    '10': 'LR618883.1',
    'MT': 'AY506529.1',
    'Pltd': 'X86563.2'
}

for chr_name, acc_id in chr_accessions.items():
    fasta_path = os.path.join(fasta_dir, f'chr_{chr_name}.fasta')
    
    # 1. Verifica se o arquivo JÁ FOI BAIXADO para não baixar de novo
    if not os.path.exists(fasta_path):
        print(f"Baixando: Cromossomo {chr_name} (ID: {acc_id})...")
        try:
            # 2. Busca (fetch) o registro no banco "nucleotide"
            handle = Entrez.efetch(db="nucleotide", id=acc_id, rettype="fasta", retmode="text")
            fasta_content = handle.read()
            handle.close()
            
            # 3. Salva o conteúdo em um arquivo .fasta
            with open(fasta_path, "w") as f:
                f.write(fasta_content)
            print(f"-> Salvo em: {fasta_path}")
            
        except Exception as e:
            print(f"ERRO ao baixar {acc_id}: {e}")
    else:
        print(f"Já existe: Cromossomo {chr_name} (Arquivo: {fasta_path})")

print("Download dos genomas concluído.")


Iniciando Fase 2: Baixando genomas do NCBI...
Já existe: Cromossomo 1 (Arquivo: ../data/fasta_genomas/chr_1.fasta)
Já existe: Cromossomo 2 (Arquivo: ../data/fasta_genomas/chr_2.fasta)
Já existe: Cromossomo 3 (Arquivo: ../data/fasta_genomas/chr_3.fasta)
Já existe: Cromossomo 4 (Arquivo: ../data/fasta_genomas/chr_4.fasta)
Já existe: Cromossomo 5 (Arquivo: ../data/fasta_genomas/chr_5.fasta)
Já existe: Cromossomo 6 (Arquivo: ../data/fasta_genomas/chr_6.fasta)
Já existe: Cromossomo 7 (Arquivo: ../data/fasta_genomas/chr_7.fasta)
Já existe: Cromossomo 8 (Arquivo: ../data/fasta_genomas/chr_8.fasta)
Já existe: Cromossomo 9 (Arquivo: ../data/fasta_genomas/chr_9.fasta)
Já existe: Cromossomo 10 (Arquivo: ../data/fasta_genomas/chr_10.fasta)
Já existe: Cromossomo MT (Arquivo: ../data/fasta_genomas/chr_MT.fasta)
Já existe: Cromossomo Pltd (Arquivo: ../data/fasta_genomas/chr_Pltd.fasta)
Download dos genomas concluído.


# Fase 3 - Carregar Genomas (BioPython.SeqIO)

In [15]:
print("\nIniciando Fase 3: Carregando genomas (FASTA) para a memória...")

genomes = {}

for chr_name in chr_accessions.keys():
    fasta_path = os.path.join(fasta_dir, f'chr_{chr_name}.fasta')
    
    try:
        # Usamos SeqIO.read() pois cada arquivo tem APENAS UMA sequência
        record = SeqIO.read(fasta_path, "fasta")
        
        # Armazenamos o objeto Seq (a sequência) no dicionário
        # A chave é o nome do cromossomo (ex: '1', 'MT')
        genomes[chr_name] = record.seq
        
        print(f"Carregado: chr {chr_name} ({len(record.seq)} bp)")
    except Exception as e:
        print(f"ERRO ao ler o arquivo {fasta_path}: {e}")

print("Genomas carregados.")


Iniciando Fase 3: Carregando genomas (FASTA) para a memória...
Carregado: chr 1 (308452471 bp)
Carregado: chr 2 (243675191 bp)
Carregado: chr 3 (238017767 bp)
Carregado: chr 4 (250330460 bp)
Carregado: chr 5 (226353449 bp)
Carregado: chr 6 (181357234 bp)
Carregado: chr 7 (185808916 bp)
Carregado: chr 8 (182411202 bp)
Carregado: chr 9 (163004744 bp)
Carregado: chr 10 (152435371 bp)
Carregado: chr MT (569630 bp)
Carregado: chr Pltd (140384 bp)
Genomas carregados.


# Fase 4: Extrair Sequências (Pandas + Bio.Seq)

In [21]:
print("\nIniciando Fase 4: Extraindo sequências de TE...")

def extrair_sequencia(linha):
    """
    Pega uma linha do DataFrame e extrai a sequência de DNA
    do dicionário de genomas.
    """
    try:
        # 1. Pega os dados da linha
        chr_name = str(linha['seqid']).replace('Chr', '').replace('chr', '')
        
        # CORREÇÃO *
        # "Traduz" os nomes do GFF para os nomes dos arquivos FASTA
        if chr_name == 'Mt':
            chr_name = 'MT'     # Se ler 'Mt', procure por 'MT'
        elif chr_name == 'Pt':
            chr_name = 'Pltd'   # Se ler 'Pt', procure por 'Pltd'
        
        
        start = int(linha['start'])
        end = int(linha['end'])
        
        # 2. Busca a sequência completa do cromossomo
        cromossomo_inteiro = genomes[chr_name]
        
        # 3. Fatiar (slice) a sequência
        # Coordenadas GFF/FASTA são 1-based (começam em 1).
        # Slices de Python são 0-based (começam em 0).
        # Por isso, usamos [start-1:end].
        te_seq = cromossomo_inteiro[start-1:end]
        
        # 4. Retorna a sequência como texto
        return str(te_seq)
        
    except KeyError:
        # Caso o seqid (ex: 'Pt') não esteja no nosso mapa de downloads
        return f"ERRO: Cromossomo '{chr_name}' não encontrado nos arquivos FASTA."
    except Exception as e:
        return f"ERRO: {e}"

# Aplica a função para cada linha do DataFrame.
# Isso pode demorar alguns minutos se o DataFrame for grande!
coords_df['Sequência de TE'] = coords_df.apply(extrair_sequencia, axis=1)

print("Extração de sequências concluída.")
print(coords_df.head())


Iniciando Fase 4: Extraindo sequências de TE...
Extração de sequências concluída.
  seqid      start        end    Classe  \
0     7   82856122   82857584  Helitron   
1     1  239302471  239302834  Helitron   
2     8    2261318    2261604  Helitron   
3     5  124665404  124665608  Helitron   
4     9   77998293   78002783  Helitron   

                                     Sequência de TE  
0  GAGCTTCGTCACCAGCTTTGCTCCGACCACCCTTTGTCCATACTAA...  
1  GTCAGGGTTGCTTCTTGGCGAAGACAGGGCCTCGGGCGAGCCAGAA...  
2  ACGCCCAAGCAGACGGTCACCATCAGCGAAGACCTCACTTCGCATG...  
3  TATGCCAAGTCGTGTCAAACGACTTAGGGTAGGGGTCAACTTTCTC...  
4  GTTAGGTTATTTATATACTAGTTTATGTTGATGATATAATCATCAC...  


# Fase 5: Salvar Dataset Final

In [22]:
print("\nIniciando Fase 5: Gerando arquivo final...")

# 1. Renomear a coluna 'seqid' para 'Cromossomo'
final_df = coords_df.rename(columns={'seqid': 'Cromossomo'})

# 2. Selecionar apenas as colunas FINAIS pedidas 
colunas_finais = ['Cromossomo', 'Sequência de TE', 'Classe']
final_df = final_df[colunas_finais]

# 3. Salvar o arquivo final
final_df.to_csv(output_dataset, index=False)

print("---")
print(f"TRABALHO CONCLUÍDO! (Etapa de montagem de dados)")
print(f"Dataset final salvo em: {output_dataset}")
print(final_df.head())


Iniciando Fase 5: Gerando arquivo final...
---
TRABALHO CONCLUÍDO! (Etapa de montagem de dados)
Dataset final salvo em: ../data/TE_dataset_final.csv
  Cromossomo                                    Sequência de TE    Classe
0          7  GAGCTTCGTCACCAGCTTTGCTCCGACCACCCTTTGTCCATACTAA...  Helitron
1          1  GTCAGGGTTGCTTCTTGGCGAAGACAGGGCCTCGGGCGAGCCAGAA...  Helitron
2          8  ACGCCCAAGCAGACGGTCACCATCAGCGAAGACCTCACTTCGCATG...  Helitron
3          5  TATGCCAAGTCGTGTCAAACGACTTAGGGTAGGGGTCAACTTTCTC...  Helitron
4          9  GTTAGGTTATTTATATACTAGTTTATGTTGATGATATAATCATCAC...  Helitron


# Fase 6: Verificação de Erros

In [23]:
print("--- Verificação de Erros ---")

# 1. Filtra o DataFrame para encontrar linhas onde 'Sequência de TE' começa com 'ERRO'
erros_df = final_df[final_df['Sequência de TE'].str.startswith('ERRO')]

# 2. Conta o número total de erros
num_erros = len(erros_df)

if num_erros == 0:
    print("Nenhuma linha com erro encontrada. Tudo certo!")
else:
    print(f"Atenção! Foram encontradas {num_erros} linhas com erro.")
    print("---")
    
    # 3. Mostra um resumo dos tipos de erro e suas contagens
    print("Resumo dos tipos de erro:")
    resumo_erros = erros_df['Sequência de TE'].value_counts()
    print(resumo_erros)
    print("---")
    
    # 4. Se houver poucos erros (ex: menos de 50), imprime o DataFrame de erros completo
    if num_erros < 50:
        print("DataFrame completo dos erros:")
        print(erros_df)
    else:
        print(f"Para ver todas as {num_erros} linhas de erro, rode o seguinte comando em uma nova célula:")
        print("print(erros_df.to_string())")

print(erros_df.to_string())

--- Verificação de Erros ---
Atenção! Foram encontradas 3270 linhas com erro.
---
Resumo dos tipos de erro:
Sequência de TE
ERRO: Cromossomo 'B73V4_ctg150' não encontrado nos arquivos FASTA.    228
ERRO: Cromossomo 'B73V4_ctg182' não encontrado nos arquivos FASTA.    177
ERRO: Cromossomo 'B73V4_ctg10' não encontrado nos arquivos FASTA.     138
ERRO: Cromossomo 'B73V4_ctg31' não encontrado nos arquivos FASTA.     111
ERRO: Cromossomo 'B73V4_ctg52' não encontrado nos arquivos FASTA.     106
                                                                     ... 
ERRO: Cromossomo 'B73V4_ctg47' não encontrado nos arquivos FASTA.       1
ERRO: Cromossomo 'B73V4_ctg246' não encontrado nos arquivos FASTA.      1
ERRO: Cromossomo 'B73V4_ctg228' não encontrado nos arquivos FASTA.      1
ERRO: Cromossomo 'B73V4_ctg172' não encontrado nos arquivos FASTA.      1
ERRO: Cromossomo 'B73V4_ctg160' não encontrado nos arquivos FASTA.      1
Name: count, Length: 239, dtype: int64
---
Para ver todas as 3

## Verificação Final dos Erros

In [ ]:
df_erros_final = final_df[final_df['Sequência de TE'].str.startswith('ERRO')]

if len(df_erros_final) == 0:
    print("Nenhum erro encontrado com os cromossomos 'Mt' e 'Pt'.")
else:
    print(f"Total de {len(df_erros_final)} erros.")
    
    # 2. Pega todos os nomes de cromossomos ÚNICOS que deram erro
    cromossomos_com_erro = df_erros_final['Cromossomo'].unique()
    
    print("\nNomes únicos dos 'Cromossomos' que causaram erro:")
    print(cromossomos_com_erro)

    # 3. Verificação automática
    # Verifica se TODOS os nomes com erro são contigs
    is_all_contigs = all(name.startswith('B73V4_ctg') for name in cromossomos_com_erro)
    
    if is_all_contigs:
        print("\nCONFIRMADO: Todos os erros restantes são de 'contigs' (B73V4_ctg...).")
    else:
        print("\nATENÇÃO: Ainda existem outros tipos de erro. Verifique a lista acima.")

Total de 3270 erros restantes.

Nomes únicos dos 'Cromossomos' que causaram erro:
['B73V4_ctg181' 'B73V4_ctg150' 'B73V4_ctg89' 'B73V4_ctg147' 'B73V4_ctg91'
 'B73V4_ctg62' 'B73V4_ctg193' 'B73V4_ctg54' 'B73V4_ctg188' 'B73V4_ctg97'
 'B73V4_ctg129' 'B73V4_ctg142' 'B73V4_ctg107' 'B73V4_ctg88' 'B73V4_ctg56'
 'B73V4_ctg183' 'B73V4_ctg178' 'B73V4_ctg40' 'B73V4_ctg31' 'B73V4_ctg102'
 'B73V4_ctg112' 'B73V4_ctg65' 'B73V4_ctg10' 'B73V4_ctg23' 'B73V4_ctg26'
 'B73V4_ctg13' 'B73V4_ctg68' 'B73V4_ctg49' 'B73V4_ctg213' 'B73V4_ctg148'
 'B73V4_ctg203' 'B73V4_ctg182' 'B73V4_ctg50' 'B73V4_ctg6' 'B73V4_ctg64'
 'B73V4_ctg52' 'B73V4_ctg42' 'B73V4_ctg53' 'B73V4_ctg84' 'B73V4_ctg133'
 'B73V4_ctg134' 'B73V4_ctg3' 'B73V4_ctg81' 'B73V4_ctg98' 'B73V4_ctg191'
 'B73V4_ctg140' 'B73V4_ctg5' 'B73V4_ctg77' 'B73V4_ctg180' 'B73V4_ctg28'
 'B73V4_ctg14' 'B73V4_ctg115' 'B73V4_ctg78' 'B73V4_ctg41' 'B73V4_ctg75'
 'B73V4_ctg51' 'B73V4_ctg170' 'B73V4_ctg29' 'B73V4_ctg190' 'B73V4_ctg151'
 'B73V4_ctg109' 'B73V4_ctg153' 'B73V4_ctg73'

Para este projeto, restringiremos nossa análise aos Elementos Transponíveis localizados nos cromossomos principais e organelares (Chr1-10, MT, Pltd), que representam a porção mais bem montada e anotada do genoma do Zea mays.

## Retirando os erros ocasionados pelas falhas por tentar obter cromossomos contíguos nos arquivos FASTA

In [25]:
total_original = len(final_df)
print(f"Número total de linhas (antes da limpeza): {total_original}")

# Cria o novo DataFrame 'df_limpo'
df_limpo = final_df[~final_df['Sequência de TE'].str.startswith('ERRO')].copy()

# Informa o resultado da limpeza
total_limpo = len(df_limpo)
total_erros = total_original - total_limpo

print(f"Linhas com erro (contigs) removidas: {total_erros}")
print(f"Número de sequências válidas (dataset limpo): {total_limpo}")
print("---")
print("Dataset limpo criado com sucesso!")

print(df_limpo.head())

Número total de linhas (antes da limpeza): 341712
Linhas com erro (contigs) removidas: 3270
Número de sequências válidas (dataset limpo): 338442
---
Dataset limpo criado com sucesso!
  Cromossomo                                    Sequência de TE    Classe
0          7  GAGCTTCGTCACCAGCTTTGCTCCGACCACCCTTTGTCCATACTAA...  Helitron
1          1  GTCAGGGTTGCTTCTTGGCGAAGACAGGGCCTCGGGCGAGCCAGAA...  Helitron
2          8  ACGCCCAAGCAGACGGTCACCATCAGCGAAGACCTCACTTCGCATG...  Helitron
3          5  TATGCCAAGTCGTGTCAAACGACTTAGGGTAGGGGTCAACTTTCTC...  Helitron
4          9  GTTAGGTTATTTATATACTAGTTTATGTTGATGATATAATCATCAC...  Helitron


In [26]:
# Agora, vamos sobrescrever o arquivo original com esta versão limpa.
print(f"\nSalvando dataset limpo em: {output_dataset}")
df_limpo.to_csv(output_dataset, index=False)
print("Arquivo final salvo (sem erros) com sucesso!")
# -----------------------------


Salvando dataset limpo em: ../data/TE_dataset_final.csv
Arquivo final salvo (sem erros) com sucesso!
